In [ ]:
##On CLI under sudo
#!pip3 install 'git+https://github.com/Troxid/vrep-api-python'

In [ ]:
import os

if "VREP_VM" in os.environ:
    os.environ["VREP"]='/opt/V-REP_PRO_EDU_V3_4_0_Linux'
    os.environ["VREP_LIBRARY"]=os.environ["VREP"]+'/programming/remoteApiBindings/lib/lib/64Bit/'
else:
    os.environ["VREP"]='/Applications/V-REP_PRO_EDU_V3_4_0_Mac'
    os.environ["VREP_LIBRARY"]=os.environ["VREP"]+'/programming/remoteApiBindings/lib/lib/'